<p style="font-weight:bold;"> <span style="font-size: 36px"> Quota Share Reinsurance</span> </p>

<p style="font-weight:bold;"> <span style="font-size: 21px"> Loss Ratio analysis</span> </p>

# Case Study

## Modelling Cash-flow inputs

A simple Insurance Contract with New Business and End Of Period runs is modeled over one year period with a Loss Ratio (LR) of 60%. We modeled the cashflows evenly distributed for both premium (1000 CHF total), Claims, and Expenses (estimated to be 30% of Premiums).

A Reinsurance Contract covers the above mentioned Insurance Contract. A Quota Share of 30% is considered and a Commission of 35% on the Reinsurance Premiums is modeled. 

Actual values entered in match the expected values. 

All these parameters are summarized in the following tables: 
<br/><br/>
|    Insurance Contract |      |
|-----------------------|------|
| Total Premium (CHF)   | 1000 |
| Internal Expense      | 30%  |
<br/>

|  Reinsurance Contract |      |
|-----------------------|------|
| Quota Share           | 30%  |
| Commission            | 35%  |




## Scenarios 

The inputs descibed in [Modelling Cash-flow inputs](#modelling-cash-flow-inputs) defines the so called "Best Estimate" Scenario. This corresponds to what is expected to happen in reality. A scenario analysis is required by IFRS 17 Standards and it can be useful in many cases to analyse risks and take decisions on the future. For example one can easily study how the profit of the Insurance and Reinsuance Contracts modelled with the above mentioned parameter would perform under different values of Loss Ratio. 
<br/><br/>
<a name="TOC"></a>
<center><img src="../../Images/QuotaShareReinsurance.png" width="45%" style="float:center;">
<br/><br/><br/>
In the best estimate scenario (Loss Ratio 60%) we expect the Insurance and Reinsurance Group Contracts to be profitable but the costs of having a Reinsurance Contract lower the total profit that the Insurance Group of Contract realizes. At Loss Ratio of 70% the Group of Insurance Contracts is very close to zero profit and raising futher the Loss Ratio to 80% a Loss making scenario is obtained where the effect of the Reinsurance contract clearly mitagates the losses as shown by the Net view. 
<br/><br/>
In the current use case, we analyse the impact of the Loss Ratio on the relevant figures computed under IFRS 17 using our standard solution implementation.
We have defined four scenarios in addition to the best estimate. We considered variations to the Loss Ratio up to 70% and 80%. In addition, we also studied the cases of having modelled the input cash flow assuming a Loss Ratio of 60% but experience the Actual cash flow having a Loss Ratio of 70% and 80%. 
<br/><br/>

|    Scenario                  | Expected Loss Ratio | Actual Loss Ratio  |
|------------------------------|---------------------|--------------------|
| Best Estimate                |  60%                | as expected        |
| Loss Ratio 70                |  70%                | as expected        |
| Loss Ratio 80                |  80%                | as expected        |
| Expected LR 60  Actual LR 70 |  60%                | 70%                |
| Expected LR 60  Actual LR 80 |  60%                | 80%                |



# Data Import

In [0]:
#!eval-notebook "DataImport"
Workspace.InitializeFrom(DataSource);

In [0]:
ifrs17 = new Ifrs17(Workspace, Scopes, Report);

In [0]:
var reportingYear = 2020; 
var reportingMonth = 12;

# Best Estimate



In [0]:
var pv = ifrs17.PresentValues;
pv.ReportingNode = "CH";
pv.ReportingPeriod = (reportingYear, reportingMonth);
pv.CurrencyType = CurrencyType.Contractual;
pv.ColumnSlices = new string[]{"Scenario","GroupOfContract", "AmountType"};
pv.DataFilter = new [] {("EconomicBasis", "L"),("Scenario","!ELR60A70"),("Scenario","!ELR60A80")};
pv.Scenario = "All";// null;//"LR70";//"All";
(await pv.ToReportAsync)

# Written Actuals

Aslo the Actuals for both cases are exactly the same.

In [0]:
var writtenActual = ifrs17.WrittenActuals;
writtenActual.ReportingNode = "CH";
writtenActual.ReportingPeriod = (reportingYear, reportingMonth);
writtenActual.ColumnSlices = new string[]{"Scenario", "GroupOfContract"};
writtenActual.DataFilter =  null;//new [] {("GroupOfContract", "RP1.1")};
writtenActual.Scenario = "All";
(await writtenActual.ToReportAsync) with {Height = 400}

# Actuarial Experience Adjustment

In [0]:
var experienceAdjustments = ifrs17.ExperienceAdjustments;
experienceAdjustments.ReportingNode = "CH";
experienceAdjustments.ReportingPeriod = (reportingYear, reportingMonth);
experienceAdjustments.ColumnSlices = new string[]{"Scenario","GroupOfContract"};//"GroupOfContract", "AmountType"
experienceAdjustments.RowSlices = new string[]{"AmountType","EstimateType"};//"GroupOfContract", "AmountType"
experienceAdjustments.DataFilter = null; //new [] {("Scenario", "!LR70"),("Scenario", "!LR80")};
experienceAdjustments.Scenario = "All";
(await experienceAdjustments.ToReportAsync)

# Contractual Service Margin / Loss Component

In [0]:
var allocatedTechnicalMargins = ifrs17.AllocatedTechnicalMargins;
allocatedTechnicalMargins.ReportingNode = "CH";
allocatedTechnicalMargins.ReportingPeriod = (reportingYear, reportingMonth);
allocatedTechnicalMargins.ColumnSlices = new string[]{"Scenario", "GroupOfContract", "EstimateType"};
allocatedTechnicalMargins.DataFilter = new [] {("Scenario","!ELR60A70"),("Scenario","!ELR60A80")}; //new [] {("Scenario", "!LR70"),("Scenario", "!LR80")};
//allocatedTechnicalMargins.Scenario = null;
allocatedTechnicalMargins.Scenario = "All";
(await allocatedTechnicalMargins.ToReportAsync) with {Height = 700}

# LRC Actuarial


In [0]:
var actuarialLrc = ifrs17.ActuarialLrc;
actuarialLrc.ReportingNode = "CH";
actuarialLrc.ReportingPeriod = (reportingYear, reportingMonth);
actuarialLrc.ColumnSlices = new string[]{"Scenario","GroupOfContract","EstimateType"};
actuarialLrc.DataFilter = new [] {("Scenario","!ELR60A70"),("Scenario","!ELR60A80")};
actuarialLrc.Scenario = "All";
(await actuarialLrc.ToReportAsync) with {Height = 750}

# Financial Performance


In [0]:
var financialPerformance = ifrs17.FinancialPerformance;
financialPerformance.ReportingNode = "CH";
financialPerformance.ReportingPeriod = (reportingYear, reportingMonth);
financialPerformance.ColumnSlices = new string[]{"Scenario", "GroupOfContract"};
financialPerformance.RowSlices = new string[]{"AmountType"};
financialPerformance.DataFilter = null; //new []{("Scenario","!LR70"),("Scenario","!ELR60A70")};//new []{("Scenario","!ELR60A70"),("Scenario","!ELR60A80")};
financialPerformance.Scenario = "All";
(await financialPerformance.ToReportAsync) with { Height = 900, GroupDefaultExpanded = 3}

# Key Performance Indicators (KPIs)

- CSM NB => Value Creation 
- CSM AM => Value Creation 
- Insurance Revenue => business Growth
- TCI PnL => PnL with OCI = 0 
- Experience Variance => managing underling business

## Contractual Service Margine : value creation

### New Business

In [0]:
var csmNb = allocatedTechnicalMargins.GetDataCube().Where(x => x.VariableType == "BOP").Select(x => x.Scenario == null ? x with {Scenario = "Best Estimate"} : x);

In [0]:
await Report.ForDataCube(csmNb.ToDataCube()).WithQuerySource(Workspace)
            .SliceRowsBy("VariableType")
            .SliceColumnsBy(new[]{"Currency","Scenario"})
            .ReportGridOptions(headerColumnWidth: 300).ExecuteAsync()

### Amortization

In [0]:
var csmAM = allocatedTechnicalMargins.GetDataCube().Where(x => x.VariableType == "AM" && Math.Abs(x.Value) > 10E-5).Select(x => x.Scenario == null ? x with {Scenario = "Best Estimate"} : x);

In [0]:
await Report.ForDataCube(csmAM.ToDataCube()).WithQuerySource(Workspace)
            .SliceRowsBy("VariableType")
            .SliceColumnsBy(new[]{"Currency","Scenario","EstimateType"})
            .ReportGridOptions(headerColumnWidth: 300).ExecuteAsync()

## Financial Performance : business growth

### Total Comprehensive Income 
In absence of the Other Comprehensive Income (OCI) this corresponds to the Profit and Loss. 

In [0]:
var tci = financialPerformance.GetDataCube().Select(x => x.Scenario == null ? x with {Scenario = "Best Estimate"} : x);
var hierarchicalDimension = Workspace.ToHierarchicalDimensionCache();
await hierarchicalDimension.InitializeAsync<VariableType>();  
var pnl = tci.Where(x => hierarchicalDimension.Get<VariableType>().Ancestors(x.VariableType, includeSelf: true).Any(x => x.SystemName == "PNL"));

In [0]:
await Report.ForDataCube(pnl.ToDataCube()).WithQuerySource(Workspace)
            //.SliceRowsBy("EstimateType")
            .SliceColumnsBy(new[]{"Currency","Scenario"}) //"GroupOfContract"
            .ReportGridOptions(headerColumnWidth: 300).ExecuteAsync()

### Insurance Revenue

In [0]:
var insRev = tci.Where(x => hierarchicalDimension.Get<VariableType>().Ancestors(x.VariableType, includeSelf: true).Any(x => x.SystemName == "IR"));

In [0]:
await Report.ForDataCube(insRev.ToDataCube()).WithQuerySource(Workspace)
            //.SliceRowsBy("EstimateType")
            .SliceColumnsBy(new[]{"Currency","Scenario","GroupOfContract"})
            .ReportGridOptions(headerColumnWidth: 300).ExecuteAsync()

## Experience Variance
This indicator represent how well the expected values match the actual. It is not a financial key performance indicator rather a modelling key performance indicator, indicating how well the company expectations are met by the actual figures.

In [0]:
experienceAdjustments.DataFilter = null;
var expAdj = experienceAdjustments.GetDataCube().Where(x => x.VariableType == "CF" && Math.Abs(x.Value) > 10E-5).Select(x => x.Scenario == null ? x with {Scenario = "Best Estimate"} : x);

In [0]:
await Report.ForDataCube(expAdj.ToDataCube()).WithQuerySource(Workspace)
            .SliceRowsBy("VariableType")
            .SliceColumnsBy(new[]{"Currency","Scenario"})
            .ReportGridOptions(headerColumnWidth: 300).ExecuteAsync()